# Task 5: Model Interpretability (SHAP & LIME)

In this task, we will use model interpretability tools—SHAP (SHapley Additive exPlanations) and LIME (Local Interpretable Model-agnostic Explanations)—to explain and visualize how our NER model makes predictions. This helps build trust in the model and identify areas for improvement.

## Setup & Imports

We will install and import the necessary libraries for model interpretability, including SHAP and LIME.

In [ ]:
pip install shap lime
import shap
import lime
from lime.lime_text import LimeTextExplainer
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
import numpy as np

## Load Fine-Tuned NER Model

We will load the best-performing fine-tuned NER model from Task 4 for interpretability analysis.

In [ ]:
model_path = "./finetuned-ner-model"  
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)

# Create a Hugging Face pipeline for NER
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

## SHAP Explanation

We will use SHAP to explain the model's predictions for a sample Amharic sentence. SHAP values show the contribution of each token to the model's entity prediction.

In [ ]:
sample_text = "አዲስ ጫማ ዋጋ 2000 ብር በ Addis Ababa"

# Tokenize and get model outputs
inputs = tokenizer(sample_text, return_tensors="pt")
outputs = model(**inputs).logits.detach().numpy()

# SHAP requires a function that takes a list of texts and returns model outputs
def f(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    return logits.detach().numpy()

explainer = shap.Explainer(f, tokenizer)
shap_values = explainer([sample_text])

# Visualize SHAP values
shap.plots.text(shap_values[0])

## LIME Explanation

We will use LIME to explain the model's predictions for a sample Amharic sentence. LIME perturbs the input and shows which words are most influential for the predicted entities.

In [ ]:
class NERPredictor:
    def __init__(self, pipeline):
        self.pipeline = pipeline
        self.labels = list(set([ent['entity_group'] for ent in self.pipeline("test") if 'entity_group' in ent]))

    def predict(self, texts):
        # Returns a list of lists of probabilities for each label
        results = []
        for text in texts:
            ents = self.pipeline(text)
            # For each label, 1 if present in prediction, else 0
            pred_labels = [ent['entity_group'] for ent in ents]
            results.append([1 if label in pred_labels else 0 for label in self.labels])
        return np.array(results)

ner_predictor = NERPredictor(ner_pipeline)
explainer = LimeTextExplainer(class_names=ner_predictor.labels)

exp = explainer.explain_instance(
    sample_text,
    ner_predictor.predict,
    num_features=10,
    labels=[0]  # Index of the label to explain
)
exp.show_in_notebook(text=True)

## Discussion

By using SHAP and LIME, we gain insights into which tokens most influence the NER model's predictions. This helps us understand model behavior, identify potential biases, and improve model trustworthiness for EthioMart's business use case.